<a href="https://colab.research.google.com/github/Vitor104/ads-machineLearningQ8/blob/main/Q8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Uma plataforma de streaming deseja sugerir filmes para os usuários com base nas avaliações de
outros usuários.
Tarefas:
- Utilize um dataset de avaliações de filmes (exemplo: MovieLens).
- Implemente um modelo de filtragem colaborativa baseado em usuários e itens.
- Compare a filtragem colaborativa com abordagens baseadas em aprendizado profundo (exemplo:
Autoencoders).
- Avalie o desempenho com métricas como RMSE e MAE.
Pergunta: Qual abordagem foi mais eficiente na recomendação de filmes? Como melhorar o
sistema de recomendação?

# 1. Importar as bibliotecas necessárias

In [27]:
# Importar as bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import NMF
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

In [13]:
# Carregar os datasets
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')

In [14]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


# Juntar os datasets

In [35]:
df = pd.merge(ratings_df, movies_df, on='movieId')

In [17]:
print(df.head())

   userId  movieId  rating  timestamp                        title  \
0       1        1     4.0  964982703             Toy Story (1995)   
1       1        3     4.0  964981247      Grumpier Old Men (1995)   
2       1        6     4.0  964982224                  Heat (1995)   
3       1       47     5.0  964983815  Seven (a.k.a. Se7en) (1995)   
4       1       50     5.0  964982931   Usual Suspects, The (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                               Comedy|Romance  
2                        Action|Crime|Thriller  
3                             Mystery|Thriller  
4                       Crime|Mystery|Thriller  


# Montar a tabela dos gostos

In [36]:
user_item_matrix = df.pivot_table(index='userId', columns='title', values='rating')

In [24]:
print(user_item_matrix.head())

title   '71 (2014)  'Hellboy': The Seeds of Creation (2004)  \
userId                                                        
1              NaN                                      NaN   
2              NaN                                      NaN   
3              NaN                                      NaN   
4              NaN                                      NaN   
5              NaN                                      NaN   

title   'Round Midnight (1986)  'Salem's Lot (2004)  \
userId                                                
1                          NaN                  NaN   
2                          NaN                  NaN   
3                          NaN                  NaN   
4                          NaN                  NaN   
5                          NaN                  NaN   

title   'Til There Was You (1997)  'Tis the Season for Love (2015)  \
userId                                                               
1                             Na

In [25]:
tabela_sem_nan = user_item_matrix.fillna(0)
print(tabela_sem_nan.head())

title   '71 (2014)  'Hellboy': The Seeds of Creation (2004)  \
userId                                                        
1              0.0                                      0.0   
2              0.0                                      0.0   
3              0.0                                      0.0   
4              0.0                                      0.0   
5              0.0                                      0.0   

title   'Round Midnight (1986)  'Salem's Lot (2004)  \
userId                                                
1                          0.0                  0.0   
2                          0.0                  0.0   
3                          0.0                  0.0   
4                          0.0                  0.0   
5                          0.0                  0.0   

title   'Til There Was You (1997)  'Tis the Season for Love (2015)  \
userId                                                               
1                             0.

# Normalizar as notas com o  MinMaxScaler

In [37]:
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(tabela_sem_nan)

# Construir a arquitetura do Autoencoder

In [38]:
input_dim = normalized_data.shape[1]

input_layer = Input(shape=(input_dim,))
encoder = Dense(10, activation='relu')(input_layer)
encoder = Dense(3, activation='relu')(encoder)

decoder = Dense(10, activation='relu')(encoder)
decoder = Dense(input_dim, activation='sigmoid')(decoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)

# Definindo as regras do treinamento

In [39]:
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [31]:
autoencoder.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 9719)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        97,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            33 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 9719)           │       106,909 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 204,182 (797.59 KB)

 Trainable params: 204,182 (797.59 KB)

 Non-trainable params: 0 (0.00 B)

# Treinamento do modelo

In [43]:
from enum import auto
# separar em treino e teste
X_train, X_test = train_test_split(normalized_data, test_size=0.2, random_state=42)
autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, validation_data=(X_test, X_test))

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0116 - val_loss: 0.0154
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0100 - val_loss: 0.0152
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0103 - val_loss: 0.0150
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0101 - val_loss: 0.0148
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0107 - val_loss: 0.0147
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0102 - val_loss: 0.0146
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0103 - val_loss: 0.0145
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0101 - val_loss: 0.0144
Epoch 9/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0091 - val_loss: 0.0144
Epoch 10/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0100 - val_loss: 0.0143


In [44]:
# Fazer previsões para o conjunto de testes
predictions = autoencoder.predict(X_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


# Avaliar desempenho com o RMSE

In [45]:
mse = mean_squared_error(X_test, predictions)
rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

RMSE: 0.11959998829470038


# Avaliar o desempenho com MAE

In [46]:
mae = np.mean(np.abs(X_test - predictions))
print(f'MAE: {mae}')

MAE: 0.028330434143742207


# Agora usando o NMF

In [52]:
nmf = NMF(n_components=50, init='random', random_state=42, max_iter=500)

In [53]:
nmf.fit(X_train)

NMF(init='random', max_iter=500, n_components=50, random_state=42)

In [49]:
H = nmf.components_
W_test = nmf.transform(X_test)
reconstructed_test = np.dot(W_test, H)

# Calcular RMSE

In [54]:
mse = mean_squared_error(X_test, reconstructed_test)
rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

RMSE: 0.09791685878930113


# Calcular MAE

In [51]:
mae = np.mean(np.abs(X_test - reconstructed_test))
print(f'MAE: {mae}')

MAE: 0.025083137511089114


# Qual abordagem foi mais eficiente na recomendação de filmes?

R: Ambos os modelos foram eficientes, com números baixos. Porém, na comparação entre as duas, o melhor é o que foi treinado com o NMF, com os seguintes números:
- RMSE: 0.11 X 0.09
- MAE: 0.02 X 0.02

# Como melhorar o sistema de recomendação?

R: O RMSE me mostra que meu modelo é eficiente, mas que algumas vezes ele erra com uma diferença muito grande quando comparado ao valor real. Para melhorá-lo ainda mais, seria preciso investigar esses erros e maximizar os acertos.